In [1]:
import requests
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# selenium을 백그라운드로 실행하기 위해 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [3]:
# 우리가 데이터를 가져올 웹 페이지입니다.
my_url = 'https://www.op.gg/summoner/userName=%EB%B4%84%EA%BD%83%EB%8F%99%ED%99%94%EB%A3%B0%EB%A3%A8'

# selenium이 제어할 chrome을 실행합니다.
# driver = webdriver.Chrome("/Users/namshin/chromedriver")
driver = webdriver.Chrome("C:/chromedriver_win32/chromedriver.exe", options= options)

<ipython-input-3-7f8705710227>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/chromedriver_win32/chromedriver.exe", options= options)


In [4]:
# 데이터를 가져올 페이지로 이동합니다.
driver.get(my_url)

In [5]:
# '더 보기' 버튼이 활성화되어있으면(리스트의 끝까지 도달하지 않았으면) 계속 클릭하여 이전 게임을 모두 로딩합니다.
while True:
    try:
        driver.find_element_by_css_selector('.GameMoreButton').click()
        # 게임 로딩, html 코드 변경까지 여유 시간을 1초 가집니다.
        time.sleep(1)
        
    # 에러가 나면(페이지에서 '더 보기' 버튼이 없을 경우) while문을 탈출합니다.
    except Exception as e:
        pass
        break

<ipython-input-5-523ee828ac39>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('.GameMoreButton').click()


In [6]:
# 게임이 모두 로딩된 페이지의 html 코드를 faker_total_html로 저장합니다.
my_total_html = driver.page_source

# selenium이 제어하는 크롬을 종료합니다.
driver.quit()

In [8]:
# html 형식에 맞춰 파싱(parsing; 추후 이용하기 쉽도록 쪼개기)합니다.
my_total_soup = BeautifulSoup(my_total_html, 'html.parser')

In [9]:
# 결과가 들어갈 빈 리스트를 만듭니다.
my_champions = []
my_results = []
my_killstate = []

In [10]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
my_total_games_html = my_total_soup.select('div.GameItemList div.GameItemWrap')

# 그렇게 가져온 html 코드의 element 개수 == 끝까지 로딩된 모든 게임 수 출력
total_game_len = len(my_total_games_html)
print(total_game_len)

343


In [11]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for i in range(total_game_len):    
    my_champions.append(list(my_total_games_html[i].select_one('div.ChampionName').
                                               stripped_strings)[0])
    my_results.append(list(my_total_games_html[i].select_one('div.GameStats div.GameResult').
                                               stripped_strings)[0])
    my_killstate.append(list(my_total_games_html[i].select_one('div.Stats div.CKRate.tip').
                                               stripped_strings)[0])

In [12]:
# DataFrame으로 변환 후 출력
total_df = pd.DataFrame([my_champions,
                                my_results,
                                my_killstate],
                               index = ['champion', 'result', 'killpercent']).T
total_df.head(10)

,champion,result,killpercent
0,룰루,승리,킬관여 45%
1,미스 포츈,패배,킬관여 53%
2,코그모,패배,킬관여 46%
3,베인,패배,킬관여 50%
4,진,패배,킬관여 56%
5,바루스,승리,킬관여 25%
6,이즈리얼,패배,킬관여 28%
7,진,패배,킬관여 55%
8,룰루,승리,킬관여 24%
9,진,승리,킬관여 68%


In [18]:
df2 = total_df.set_index("champion")
df2.head(10)

,result,killpercent
champion,,
룰루,승리,킬관여 45%
미스 포츈,패배,킬관여 53%
코그모,패배,킬관여 46%
베인,패배,킬관여 50%
진,패배,킬관여 56%
바루스,승리,킬관여 25%
이즈리얼,패배,킬관여 28%
진,패배,킬관여 55%
룰루,승리,킬관여 24%


In [19]:
most_champions = []
most_games = []

In [20]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
my_total_games_html = my_total_soup.select('div.MostChampionContent div.ChampionBox.Ranked')

# 그렇게 가져온 html 코드의 element 개수 == 끝까지 로딩된 모든 게임 수 출력
total_game_len = len(my_total_games_html)
print(total_game_len)

7


In [21]:
for i in range(total_game_len):    
    most_champions.append(list(my_total_games_html[i].select_one('div.ChampionInfo div.ChampionName').
                                               stripped_strings)[0])
    most_games.append(list(my_total_games_html[i].select_one('div.Played div.Title').
                                              stripped_strings)[0])

In [22]:
total_df = pd.DataFrame([most_champions,
                                most_games],
                               index = ['champion', 'games']).T
total_df

,champion,games
0,진,128 게임
1,룰루,55 게임
2,이즈리얼,54 게임
3,직스,34 게임
4,애쉬,28 게임
5,세나,28 게임
6,바루스,25 게임


In [25]:
df2 = total_df.set_index("champion")
df2

,games
champion,
진,128 게임
룰루,55 게임
이즈리얼,54 게임
직스,34 게임
애쉬,28 게임
세나,28 게임
바루스,25 게임
